In [ ]:
import os 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import librosa
import librosa.display
from time_encoder_2D import time_encoder_2D
from model import hd_model

In [ ]:
def db_melspectrogram(wav , sr , n_fft, hop_length, n_mels, fmin, fmax, top_db):
    spec=librosa.feature.melspectrogram(y=wav, sr=sr, n_fft=n_fft, hop_length=hop_length,n_mels=n_mels,fmin=fmin,fmax=fmax)
    spec_db=librosa.power_to_db(spec,top_db=top_db)
    spec_db = spec_db.astype(np.float32)
    return spec_db

In [ ]:
y, sr = librosa.load("./data/alarms/alarms01.wav", sr = None)
mel_sp = db_melspectrogram(wav = y, sr = sr, n_fft = 2048, hop_length=512, top_db = 80, n_mels = 64, fmin= 0, fmax= int(sr /2))

# img = librosa.display.specshow(mel_sp, x_axis='time',y_axis='mel')

print(mel_sp.shape)

In [ ]:
frame_amount = int (mel_sp.shape[1] / mel_sp.shape[0])
frame_amount

In [ ]:
amount_of_train = 14
amount_of_test = 4
amount_of_label = 10

In [ ]:
classified_path = "./data/"
folder_list = os.listdir(classified_path)

label = 0

hd_dim = 10000

data = np.zeros(64 * 64)

encoder = time_encoder_2D(64 ,hd_dim, 2000 , 1.0, 64, 1)

model = hd_model(10 , hd_dim)

for i in folder_list:
     
    counter = 0
    print("This is " + i) 
    sub_folder_path = classified_path + i + "/"
    sub_folder_list = os.listdir(sub_folder_path)

    for j in sub_folder_list[: amount_of_train]:
        y, sr = librosa.load(sub_folder_path + "/" + j, sr = None)
        mel_sp = db_melspectrogram(wav = y, sr = sr, n_fft = 2048, hop_length=512, top_db = 80, n_mels = 64, fmin= 0, fmax= int(sr /2))
        for filter in range(0 ,mel_sp.shape[0]):
            mel_sp[filter] = np.divide(mel_sp[filter] , np.max(np.abs(mel_sp[filter])) )
        mel_sp = np.transpose(mel_sp)
        

        for f in range(0 , frame_amount - 1):
            data = mel_sp[f * 64 : (f + 1) * 64]
            
            model.add_hdv(encoder.encode(input_data = data), str(label))
        counter = counter + 1
    
    label = label + 1

In [ ]:
model.model_data
# model.bipolar_model_data()
# model.bipolar_data

In [ ]:
# correct = 0
# tested = 0
# label = 0
# for i in folder_list:
#     print("This is " + i) 

#     sub_folder_path = classified_path + i + "/"
#     sub_folder_list = os.listdir(sub_folder_path)    

#     for j in sub_folder_list[amount_of_train:]:
#         y, sr = librosa.load(sub_folder_path + "/" + j, sr = None)
#         mel_sp = db_melspectrogram(wav = y, sr = sr, n_fft = 2048, hop_length=512, top_db = 80, n_mels = 64, fmin= 0, fmax= int(sr /2))
#         for filter in range(0 ,mel_sp.shape[0]):
#             mel_sp[filter] = np.divide(mel_sp[filter] , np.max(np.abs(mel_sp[filter])) )
            
#         mel_sp = np.transpose(mel_sp)
    
#         for f in range(0 , frame_amount - 1):
#             data = mel_sp[f * 64 : (f + 1) * 64]
#             data = data.flatten()
#             hdv = encoder.encode(input_data = data)
            
#             max_ham = 0
#             predict_label = "0"

#             for i in model.bipolar_data.keys():
#                 ham_helper = 0
#                 for index in range(0, hd_dim):
#                     if model.bipolar_data[i][index] == hdv[index]:
#                         ham_helper = ham_helper + 1
#                 if ham_helper > max_ham:
#                     max_ham = ham_helper
#                     predict_label = i
#             tested = tested + 1
#             if predict_label == str(label):
#                 correct = correct + 1
#             print(label, predict_label)

#     label = label + 1

In [ ]:
def cosine_sim(A, B):
    return np.dot(A,B)/(np.linalg.norm(A)*np.linalg.norm(B))

correct = 0
tested = 0
label = 0
for i in folder_list:
    print("This is " + i) 

    sub_folder_path = classified_path + i + "/"
    sub_folder_list = os.listdir(sub_folder_path)    

    for j in sub_folder_list[amount_of_train:]:
        y, sr = librosa.load(sub_folder_path + "/" + j, sr = None)
        mel_sp = db_melspectrogram(wav = y, sr = sr, n_fft = 2048, hop_length=512, top_db = 80, n_mels = 64, fmin= 0, fmax= int(sr /2))
        for filter in range(0 ,mel_sp.shape[0]):
            mel_sp[filter] = np.divide(mel_sp[filter] , np.max(np.abs(mel_sp[filter])) )
            
        mel_sp = np.transpose(mel_sp)
        
        label_list = [0] * amount_of_label

        for f in range(0 , frame_amount - 1):
            data = mel_sp[f * 64 : (f + 1) * 64]
            hdv = encoder.encode(input_data = data)
            
            max_cos = 0
            predict_label = "0"

            for i in model.bipolar_data.keys():
                ham_helper = cosine_sim(hdv, model.model_data[i])
                if ham_helper > max_cos:
                    max_cos = ham_helper
                    predict_label = i
            label_list[int(predict_label)] += 1

        predict_label = str(label_list.index(max(label_list)))

        tested = tested + 1
        if predict_label == str(label):
            correct = correct + 1
        print(label, predict_label)

    label = label + 1

In [ ]:
correct / tested